In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "c0f9f0e9",
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_data():\n",
    "    \n",
    "    import pandas as pd\n",
    "\n",
    "    dataframe = pd.read_csv(\n",
    "        \"../files/input/sentences.csv.zip\",\n",
    "        index_col=False,\n",
    "        compression=\"zip\",\n",
    "    )\n",
    "\n",
    "    data = dataframe.phrase\n",
    "    target = dataframe.target\n",
    "\n",
    "    return data, target"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "7dafa423",
   "metadata": {},
   "outputs": [],
   "source": [
    "def make_train_test_split(x, y):\n",
    "    \n",
    "    from sklearn.model_selection import train_test_split\n",
    "\n",
    "    (x_train, x_test, y_train, y_test) = train_test_split(\n",
    "        x,\n",
    "        y,\n",
    "        test_size=0.25,\n",
    "        random_state=123456,\n",
    "    )\n",
    "    return x_train, x_test, y_train, y_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "b9546863",
   "metadata": {},
   "outputs": [],
   "source": [
    "def make_pipeline(estimator):\n",
    "    \n",
    "    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer\n",
    "    from sklearn.pipeline import Pipeline\n",
    "\n",
    "    vectorizer = CountVectorizer(\n",
    "        lowercase=True,\n",
    "        analyzer=\"word\",\n",
    "        token_pattern=r\"\\b[a-zA-Z]\\w+\\b\",\n",
    "        stop_words=\"english\",\n",
    "    )\n",
    "\n",
    "    transformer = TfidfTransformer()\n",
    "\n",
    "    pipeline = Pipeline(\n",
    "        steps=[\n",
    "            (\"vectorizer\", vectorizer),\n",
    "            (\"transformer\", transformer),\n",
    "            (\"estimator\", estimator),\n",
    "        ],\n",
    "        verbose=False,\n",
    "    )\n",
    "\n",
    "    return pipeline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "c6ebbbc7",
   "metadata": {},
   "outputs": [],
   "source": [
    "def make_grid_search(estimator, param_grid, cv=5):\n",
    "    \n",
    "    from sklearn.model_selection import GridSearchCV\n",
    "\n",
    "    grid_search = GridSearchCV(\n",
    "        estimator=estimator,\n",
    "        param_grid=param_grid,\n",
    "        cv=cv,\n",
    "        scoring=\"balanced_accuracy\",\n",
    "    )\n",
    "\n",
    "    return grid_search"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "2f5af7aa",
   "metadata": {},
   "outputs": [],
   "source": [
    "def save_estimator(estimator):\n",
    "    \n",
    "    import pickle\n",
    "\n",
    "    with open(\"estimator.pickle\", \"wb\") as file:\n",
    "        pickle.dump(estimator, file)\n",
    "        "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "ba015657",
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_estimator():\n",
    "    \n",
    "    import os\n",
    "    import pickle\n",
    "\n",
    "    if not os.path.exists(\"estimator.pickle\"):\n",
    "        return None\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    return estimator"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "ca97cdd1",
   "metadata": {},
   "outputs": [],
   "source": [
    "def train_logistic_regression():\n",
    "    \n",
    "    from sklearn.linear_model import LogisticRegression\n",
    "    from sklearn.metrics import balanced_accuracy_score\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train, y_test = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    pipeline = make_pipeline(\n",
    "        estimator=LogisticRegression(max_iter=1000),\n",
    "    )\n",
    "\n",
    "    estimator = make_grid_search(\n",
    "        estimator=pipeline,\n",
    "        param_grid={\n",
    "            \"transformer__norm\": [\"l1\", \"l2\", None],\n",
    "            \"transformer__use_idf\": [True, False],\n",
    "            \"transformer__smooth_idf\": [True, False],\n",
    "        },\n",
    "        cv=5,\n",
    "    )\n",
    "\n",
    "    estimator.fit(x_train, y_train)\n",
    "\n",
    "    best_estimator = load_estimator()\n",
    "\n",
    "    if best_estimator is not None:\n",
    "\n",
    "        saved_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=best_estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        current_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        if current_balanced_accuracy < saved_balanced_accuracy:\n",
    "            estimator = best_estimator\n",
    "\n",
    "    save_estimator(estimator)\n",
    "\n",
    "\n",
    "train_logistic_regression()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "2932a1ba",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['neutral', 'positive', 'positive', ..., 'negative', 'negative',\n",
       "       'negative'], shape=(2264,), dtype=object)"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "def use_estimator():\n",
    "    \n",
    "    import pickle\n",
    "\n",
    "    import pandas as pd\n",
    "\n",
    "    dataframe = pd.read_csv(\n",
    "        \"../files/input/sentences.csv.zip\",\n",
    "        index_col=False,\n",
    "        compression=\"zip\",\n",
    "    )\n",
    "\n",
    "    data = dataframe.phrase\n",
    "\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    prediction = estimator.predict(data)\n",
    "\n",
    "    return prediction\n",
    "\n",
    "\n",
    "use_estimator()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "9d1a7913",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Pipeline(steps=[('vectorizer',\n",
      "                 CountVectorizer(stop_words='english',\n",
      "                                 token_pattern='\\\\b[a-zA-Z]\\\\w+\\\\b')),\n",
      "                ('transformer', TfidfTransformer(norm=None, smooth_idf=False)),\n",
      "                ('estimator', LogisticRegression(max_iter=1000))]):\n",
      "  Balanced Accuracy: 0.7165 (0.9962)\n",
      "           Accuracy: 0.8233 (0.9982)\n"
     ]
    }
   ],
   "source": [
    "def check_estimator():\n",
    "    \n",
    "    import pickle\n",
    "\n",
    "    import pandas as pd\n",
    "    from sklearn.metrics import accuracy_score, balanced_accuracy_score\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train_true, y_test_true = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    y_train_pred = estimator.predict(x_train)\n",
    "    y_test_pred = estimator.predict(x_test)\n",
    "\n",
    "    accuracy_train = round(accuracy_score(y_train_true, y_train_pred), 4)\n",
    "    accuracy_test = round(accuracy_score(y_test_true, y_test_pred), 4)\n",
    "    balanced_accuracy_train = round(\n",
    "        balanced_accuracy_score(y_train_true, y_train_pred), 4\n",
    "    )\n",
    "    balanced_accuracy_test = round(balanced_accuracy_score(y_test_true, y_test_pred), 4)\n",
    "\n",
    "    print(estimator.best_estimator_, \":\", sep=\"\")\n",
    "    print(f\"  Balanced Accuracy: {balanced_accuracy_test} ({balanced_accuracy_train})\")\n",
    "    print(f\"           Accuracy: {accuracy_test} ({accuracy_train})\")\n",
    "    \n",
    "\n",
    "check_estimator()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f91c8a53",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "c:\\Users\\Asus\\Documents\\unal\\PRE-21-pipelines-Dgarzonac9\\venv\\Lib\\site-packages\\sklearn\\neural_network\\_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.\n",
      "  warnings.warn(\"Training interrupted by user.\")\n"
     ]
    }
   ],
   "source": [
    "def train_mlp_classifier():\n",
    "    \n",
    "    from sklearn.metrics import balanced_accuracy_score\n",
    "    from sklearn.neural_network import MLPClassifier\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train, y_test = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    pipeline = make_pipeline(\n",
    "        estimator=MLPClassifier(max_iter=10000),\n",
    "    )\n",
    "\n",
    "    estimator = make_grid_search(\n",
    "        estimator=pipeline,\n",
    "        param_grid={\n",
    "            \"transformer__norm\": [\"l1\", \"l2\", None],\n",
    "            \"transformer__use_idf\": [True, False],\n",
    "            \"transformer__smooth_idf\": [True, False],\n",
    "            \"estimator__hidden_layer_sizes\": [(1,), (5,), (5, 5)],\n",
    "            \"estimator__solver\": [\"adam\"],\n",
    "            \"estimator__learning_rate_init\": [0.01, 0.001, 0.0001],\n",
    "        },\n",
    "        cv=5,\n",
    "    )\n",
    "\n",
    "    estimator.fit(x_train, y_train)\n",
    "\n",
    "    best_estimator = load_estimator()\n",
    "\n",
    "    if best_estimator is not None:\n",
    "\n",
    "        saved_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=best_estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        current_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        if current_balanced_accuracy < saved_balanced_accuracy:\n",
    "            estimator = best_estimator\n",
    "\n",
    "    save_estimator(estimator)\n",
    "\n",
    "\n",
    "train_mlp_classifier()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4d40f948",
   "metadata": {},
   "outputs": [],
   "source": [
    "check_estimator()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "venv (3.14.0)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.14.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}